# Faster RCNN for Global Wheat Detection


### Understanding Task

The task of this competition is to predict the bounding boxes of wheat heads in different images. The images have a varying number of wheat heads, colors, orientations, and so on making the task more challenging. These images are used to estimate the density and size of wheat heads in different varieties. Farmers can use the data to assess health and maturity when making management decisions in their fields.


### About Dataset

- There is a toal of 3422 unique train images. The code to get this number:
- Thus some images don't have any masks
- There is a toal of 147793 masks
- Thus, on average, there are 43.8 masks per image
- The image with the most masks contains 116. It is the image with id 35b935b6c.
- All the train images have the same size: 1024 x 1024.
- There are 3 channels: R, G, B.
- train.csv: each row show coordinates of a wheat bounding box. Information fields is: image_id, image_width, image_height, bbox(x, y, w, h)

## 1. Visualize GWD dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import os
import re

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

In [ ]:
INPUT_DIR = '/kaggle/input/global-wheat-detection'
TRAIN_DIR = f'{INPUT_DIR}/train'
TEST_DIR = f'{INPUT_DIR}/test'
TRAIN_CSV_PATH = f'{INPUT_DIR}/train.csv'

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df.head()

In [ ]:
sns.countplot(train_df['source'])

Separating x,y,w,h into separate columns for convenience

In [ ]:
bboxes = np.stack(train_df['bbox'].apply(lambda x:np.fromstring(x[1:-1], sep=',')))

for i, col in enumerate(['x', 'y', 'w', 'h']):
    train_df[col] = bboxes[:, i]

Dropping the bbox column as it is not needed now

In [ ]:

train_df['box_area'] = train_df['w']*train_df['h']

In [ ]:
train_df.head()

In [ ]:
#number of unique images in the dataframe
len(train_df['image_id'].unique())

In [ ]:
#number of images in the training directory
len(os.listdir(TRAIN_DIR))

In [ ]:
#append .jpg to image ids for easier handling
train_df['image_id'] = train_df['image_id'].apply(lambda x: str(x) + '.jpg')
train_df['image_id'].head()

In [ ]:
#obtaining a list of all images which have no wheat heads in them
unique_imgs_wbox = list(train_df['image_id'].unique())
all_unique_imgs = os.listdir(TRAIN_DIR)
no_wheat_imgs = [img_id for img_id in all_unique_imgs if img_id not in unique_imgs_wbox]
len(no_wheat_imgs)

In [ ]:
def get_all_bboxes(df, image_id, count=False):
    bboxes = []
    
    for _,row in df[df.image_id == image_id].iterrows():
        bboxes.append([row.x, row.y, row.w, row.h])
    if count:
        return bboxes, len(bboxes)
    else:
        return bboxes
    

def select_img(df, n, wheat=True):
    
    if wheat:
        img_ids = df.sample(n=n, random_state=0)['image_id']
        return list(img_ids)
    else:
        img_ids = np.random.choice(no_wheat_imgs, n)
        return list(img_ids)
        

def plot_image(df, ids, bbox=False):
    n = len(ids)
    fig, ax = plt.subplots(2, n//2, figsize=(40,30))
    
    for i, img_id in enumerate(ids):
        img = mpimg.imread(os.path.join(TRAIN_DIR, img_id))
        ax[i//(n//2)][i%(n//2)].imshow(img)
        ax[i//(n//2)][i%(n//2)].axis('off')
        
        if bbox:
            bboxes = get_all_bboxes(df, img_id)
            for bbox in bboxes:
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=2,edgecolor='r',facecolor='none')
                ax[i//(n//2)][i%(n//2)].add_patch(rect)
        else:
            pass
        
    plt.tight_layout()
    plt.show()
                

In [ ]:
plot_image(train_df, select_img(train_df,6))


In [ ]:
plot_image(train_df, select_img(train_df, 6, wheat=False))

In [ ]:
plot_image(train_df, select_img(train_df,6), bbox=True)

## 2. Create DataLoader

### Split train-valid dataframe

In [ ]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

In [ ]:
train_df.shape

In [ ]:
valid_df.shape

### Create DataLoader

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

In [ ]:
class WheatDataset(Dataset):
    
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        
        image = cv2.imread(os.path.join(self.image_dir, image_id), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB). astype(np.float32)
        image /= 255.0
        
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:,0] + boxes[:,2] #x_max = x_min(x) + w
        boxes[:, 3] = boxes[:,1] + boxes[:,3] #y_max = y_min(y) + h
        
        area = records['box_area'].values
        area = torch.as_tensor(area, dtype=torch.float32)
        
        #there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        #suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
            
        return image, target, image_id
    
    def __len__(self)->int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_train_transforms():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields':['labels']})

def get_valid_transforms():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields':['labels']})

# Data Loader
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataset = WheatDataset(train_df, TRAIN_DIR, get_train_transforms())
valid_dataset = WheatDataset(train_df, TRAIN_DIR, get_valid_transforms())

In [ ]:
# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist

In [ ]:
train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

## 3. Create Model

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 2 # wheat + no_wheat(background)

#get number of input features for the  classificatier
in_features = model.roi_heads.box_predictor.cls_score.in_features

#replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
        
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total/self.iterations
        
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

## 4. Training Model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)
params = [p for p in model.parameters() if  p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.StepLR(opotimizer, step_size=3, gamma=0.1)
num_epochs = 2

loss_hist = Averager()

itr = 1

In [ ]:
for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k,v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        loss_hist.send(loss_value)
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")
            
        itr += 1
        
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
        
    print(f"Epoch #{epoch} loss: {loss_hist.value}")

In [ ]:
images, targets, image_ids = next(iter(valid_data_loader))

In [ ]:
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

In [ ]:
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()

In [ ]:
model.eval()
cpu_device = torch.device("cpu")

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')

## 5. Object Detection with [Pytorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning)

### 5.1. Import libraries

In [ ]:
!pip uninstall pycocotools -y
!pip install -q git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI

In [ ]:
!pip install hydra-core
!pip install pytorch-lightning==0.8.1

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
from PIL import Image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
IMG_SIZE = 256

In [ ]:
def get_train_transforms():
    return A.Compose([
        A.RandomSizedCrop(min_max_height=(800,800), height=IMG_SIZE, width=IMG_SIZE, p=0.5),
        A.OneOf([
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2,
                                val_shift_limit=0.2, p=0.9),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9),
        ], p=0.9),
        A.ToGray(p=0.01),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Resize(height=256, width=256, p=1),
        A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
        ToTensorV2(p=1.0),
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format='pascal_voc',
        min_area=0,
        min_visibility=0,
        label_fields=['labels']
        )
    )

In [ ]:
def get_valid_transforms():
    return A.Compose([
        A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
        ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
def get_test_transforms():
    return A.Compose([
        A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.0)

In [ ]:
class WheatDataset(Dataset):
    
    def __init__(self, df=None, mode="train", image_dir="", transforms=None):
        super().__init__()
        if df is not None:
            self.df = df.copy()
            self.image_ids = df['image_id'].unique()
        else:
            # test case
            self.df = None
            self.image_ids = [p.stem for p in Path(image_dir).glob("*.jpg")]
        
        self.image_dir = image_dir
        self.transforms = transforms
        self.mode = mode
        
        
    def __getitem__(self, index:int):
        image_id = self.image_ids[index]
        
        image = Image.open(f'{self.image_dir}/{image_id}').convert("RGB")
        image = np.array(image)
        image = image/255.
        image = image.astype(np.float32)
        
        if self.mode != 'test':
            records = self.df[self.df['image_id'] == image_id]
            
            area = records['box_area'].values
            area = torch.as_tensor(area, dtype=torch.float32)
            
            boxes = records[['x', 'y', 'w', 'h']].values
            boxes[:, 2] = boxes[:,0] + boxes[:,2] #x_max = x_min(x) + w
            boxes[:, 3] = boxes[:,1] + boxes[:,3] #y_max = y_min(y) + h
            
            labels = torch.ones((records.shape[0],), dtype=torch.int64)
            
            iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
            
            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            # target['masks'] = None
            target['image_id'] = torch.tensor([index])
            target['area'] = area
            target['iscrowd'] = iscrowd
            # These are needed as well by the efficientdet model.
            target['img_size'] = torch.tensor([(IMG_SIZE, IMG_SIZE)])
            target['img_scale'] = torch.tensor([1.])
            
        else:
            target = {'cls': torch.as_tensor([[0]], dtype=torch.float32),
                      'bbox': torch.as_tensor([[0,0,0,0]], dtype=torch.float32),
                      'img_size': torch.tensor([(IMG_SIZE, IMG_SIZE)]),
                      'img_scale': torch.tensor([1.])
                     }
            
        
        if self.mode != 'test':
            
            if self.transforms:
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                }
                if len(sample['bboxes']) > 0:
                    #apply augmentation on the fly
                    sample = self.transforms(**sample)
                    image = sample['image']
                    boxes = sample['bboxes']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*boxes)))).permute(1,0)
            
            else:
                sample = {
                    'image': image,
                    'bbox': target['bbox'],
                    'cls': target['cls']
                }
                image = self.transforms(**sample)['image']
        
            return image, target
        
    def __len__(self) -> int:
        return len(self.image_ids)

In [ ]:
processed_train_labels_df = train_df.copy()
processed_train_labels_df["x2"] = processed_train_labels_df["x"] + processed_train_labels_df["w"]
processed_train_labels_df["y2"] = processed_train_labels_df["y"] + processed_train_labels_df["h"]

In [ ]:
processed_train_labels_df.head()

In [ ]:
# Create stratified folds, here using the source.
# This isn't the most optimal way to do it but I will leave it to you 
# to find a better one. 
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
for fold, (train_index, valid_index) in enumerate(skf.split(processed_train_labels_df,
                                                            y=processed_train_labels_df['source'])):
    processed_train_labels_df.loc[valid_index, 'fold'] = fold

In [ ]:
processed_train_labels_df.sample(2).T

In [ ]:
train_transforms = get_train_transforms()

In [ ]:
train_dataset = WheatDataset(processed_train_labels_df, mode='train',
                            image_dir=TRAIN_DIR, transforms=train_transforms)

In [ ]:
image, target = train_dataset[0]

In [ ]:
image

In [ ]:
target

In [ ]:
processed_train_labels_df.head()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pandas as pd
from pathlib import Path

def plot_image_with_bboxes(img_id, df):
    img_path= Path(f'{TRAIN_DIR}/{img_id}')
    img = Image.open(img_path)
    draw = ImageDraw.Draw(img)
    
    bboxes=[]
    for _,row in df[df.image_id == img_id].iterrows():
        bboxes.append([row.x, row.y, row.w, row.h])
        
    for bbox in bboxes:
        x, y, w, h = bbox
#         print(x, y, w, h)
        transformed_bbox = [x, y, x + w, y + h]
        draw.rectangle(transformed_bbox, outline="red", width=3)
        
    return img

In [ ]:
plot_image_with_bboxes('b6ab77fd7.jpg', train_df)

### 5.2 Pytorch Lightning Complete Pipline

**Pytorch Lightning**: we will build a model, start with the **model building block** then addd the **processing** step.

Fisetly, we install the nesscessary library:

In [ ]:
!pip install --upgrade pip
!pip install pytorch_lightning
!pip install effdet --upgrade
!pip install timm
!pip install omegaconf
!pip install pycocotools

Create the EfficientDet model using the code snippet presented above: the new `create_model` code snippet instead of the `get_train_efficientdet` function since we are using the latest effdet version

In [ ]:
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet


def get_train_efficientdet():
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load('../input/efficientdet/efficientdet_d5-ef44aea8.pth')
    net.load_state_dict(checkpoint)
    config.num_classes = 1 
    config.image_size = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=0.001, momentum=0.01))
    return DetBenchTrain(net, config)

In [ ]:
# new way
from effdet import create_model

def get_train_efficientdet():
    return create_model('tf_efficientdet_d5', bench_task='train', 
                        num_classes=2, bench_labeler=True)

In [ ]:
from pytorch_lightning import LightningModule

class WheatModel(LightningModule):
    
    def __init__(self,  df, fold):
        super().__init__()
        self.df = df
        self.train_df = self.df.loc[lambda df: df["fold"] != fold]
        self.valid_df = self.df.loc[lambda df: df["fold"] == fold]
        self.image_dir = TRAIN_DIR
        self.model = get_train_efficientdet()
        self.num_workers = 4
        self.batch_size = 8
    
    def forward(self, image, target):
        return self.model(image, target)
    
#Create model for one fold
model = WheatModel(processed_train_labels_df, fold=0)
    

In [ ]:
model

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

def collate_fn(batch):
    return tuple(zip(*batch))

# add the train and validation data loaders
def train_dataloader(self):
    train_transforms = get_train_transforms()
    train_dataset = WheatDataset(self.train_df, image_dir=self.image_dir,
                                transforms=train_transforms)
    return DataLoader(
        train_dataset,
        batch_size=self.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        collate_fn=collate_fn,
        num_workers=self.num_workers,
    )

def val_dataloader(self):
    valid_transforms = get_train_transforms()
    valid_dataset = WheatDataset(self.valid_df, image_dir=self.image_dir,
                                transforms=valid_transforms)
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=self.batch_size,
        sampler=SequentialSampler(valid_dataset),
        pin_memory=False,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=self.num_workers,
    )
    
    iou_types = ["bbox"]
    
    return valid_dataloader

In [ ]:
WheatModel.train_dataloader = train_dataloader
WheatModel.val_dataloader = val_dataloader

In [ ]:
def training_step(self, batch, batch_idx):
    images, targets = batch
    targets = [{k: v for k, v in t.items()} for t in targets]
    
    #separate losses
    images = torch.stack(images).float()
    targets2 = {}
    targets2["bbox"] = [target["boxes"].float() for target in targets]
    #variable nu,ber of instances, so the entire structure can be forced to tensor
    targets2["cls"] = [target["labels"].float() for target in targets]
    """
    targets2["image_id"] = torch.tensor(
        [target["image_id"] for target in targets]
    ).float()
    targets2["img_scale"] = torch.tensor(
        [target["img_scale"] for target in targets], device="cuda"
    ).float()
    targets2["img_size"] = torch.tensor(
        [(IMG_SIZE, IMG_SIZE) for target in targets], device="cuda"
    ).float()
    """
    loses_dict = self.model(images, targets2)
    
    return {"loss": losses_dict["loss"], "log": losses_dict}


In [ ]:
def validation_step(self, batch, batch_idx):
    images, targets = batch
    targets = [{k: v for k, v in t.items()} for t in targets]
    
    #separate losses
    images = torch.stack(images).float()
    targets2 = {}
    targets2["bbox"] = [target["boxes"].float() for target in targets]
    #variable nu,ber of instances, so the entire structure can be forced to tensor
    targets2["cls"] = [target["labels"].float() for target in targets]
    """
    targets2["image_id"] = torch.tensor(
        [target["image_id"] for target in targets]
    ).float()
    targets2["img_scale"] = torch.tensor(
        [target["img_scale"] for target in targets], device="cuda"
    ).float()
    targets2["img_size"] = torch.tensor(
        [(IMG_SIZE, IMG_SIZE) for target in targets], device="cuda"
    ).float()
    """
    loses_dict = self.model(images, targets2)
    loss_val = losses_dict["loss"]
    detections = losses_dict["detections"]
    #Back to x, y, x, y format
    detections[:,:,[1,0,3,2]] = detections[:, :, [0,1,2,3]]
    
    res = {target["image_id"].item(): {
                'boxes': output[:, 0:4],
                'scores': output[:, 4],
                'labels': output[:, 5]      
    }for target, output in zip(targets, detections)}
    
    # iou = self._calculate_iou(targets, res, IMG_SIZE)
    # iou = torch.as_tensor(iou)
    # self.coco_evaluator.update(res)
    
    return {"loss": losses_dict["loss"], "log": losses_dict}

In [ ]:
def validation_epoch_end(self, outputs):
    # self.coco_evaluator.accumulate()
    # self.coco_evaluator.summarize()
    # coco main metric
    # metric = self.coco_evaluator.coco_eval["bbox"].stats[0]
    # metric = torch.as_tensor(metric)
    # tensorboard_logs = {"main_score": metric}
    # return {
    #     "val_loss": metric,
    #     "log": tensorboard_logs,
    #     "progress_bar": tensorboard_logs,
    # }
    pass

def configure_optimizers(self):
    return torch.optim.AdamW(self.model.parameters(), lr=1e-4)

In [ ]:
WheatModel.training_step = training_step
# WheatModel.validation_step = validation_step
# WheatModel.validation_epoch_end = validation_epoch_end
WheatModel.configure_optimizers = configure_optimizers

`WheatModel` is ready now. Let's train it. For that, we will create a `Trainer` and set it to `fast_dev_run=True` for a quicker demo. Also, since it is in this mode, the Trainer doesn't automatically save the weights at the end (correct me if I am wrong of course) so we need to add a torch.save call at the end.

In [ ]:
from pytorch_lightning import Trainer, seed_everything, loggers

seed_everything(314)

#create model for one fold
model = WheatModel(processed_train_labels_df, fold=0)
logger = loggers.TensorBoardLogger("logs", name="effdet-b5", version="fold_0")
trainer = Trainer(gpus=1, logger=logger, fast_dev_run=True)
trainer.fit(model)
torch.save(model.model.state_dict(), "wheatdet.pth")